In [6]:
import numpy as np
import json
import uuid
import os

from keras.layers import Convolution2D, Activation, MaxPooling2D, Dropout, Flatten, Dense, ZeroPadding2D, Lambda, ELU, BatchNormalization
from keras.models import Sequential, model_from_json
from keras.optimizers import Adam

In [ ]:
class TrainingDataset:
    def load(self, validation_split_percentage=0.01):
        X_train, y_train = [], []

        # Only look at latest driving_log.csv
        drive_log_path = './driving_log.csv'

        if os.path.isfile(drive_log_path):
            df = pd.read_csv(drive_log_path)
            headers = list(df.columns.values)
            print(headers)
            for index, row in df.iterrows():
                # print(observation)
                c = row['center'].strip()
                l = row['left'].strip()
                r = row['right'].strip()
                a = float(row['steering'])

                if os.path.isfile(c):
                    # casts absolute path to relative to remain env agnostic
                    l, c, r = [('IMG/' + os.path.split(file_path)[1]) for file_path in (l, c, r)]
                    # single string in memory
                    x = '{}:{}:{}'.format(l, c, r)
                    X_train.append(x)
                    y_train.append(a)

        # Split some of the training data into a validation dataset
        X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=validation_split_percentage,
            random_state=0)

        X_train, y_train, X_val, y_val = np.array(X_train), np.array(y_train, dtype=np.float32), np.array(X_val), np.array(
            y_val, dtype=np.float32)
        
        self.X_train = X_train
        self.X_val = X_val
        self.y_train = y_train
        self.y_val = y_val
        
        return X_train, y_train, X_val, y_val

In [9]:
class BaseNetwork:
    
    WEIGHTS_FILE_NAME = 'model_final.h5'
    MODEL_FILE_NAME = 'model_final.json'
    
    def __init__(self):
        self.uuid = uuid.uuid4()
        self.model = None
        self.weights = None
        self.__configured = True

    def fit(self, X_train, y_train, nb_epoch=12, batch_size=128, validation_data=None, shuffle=True):
        raise NotImplementedError

    def save(self):
        print('Saved {} model.'.format(self.__class__.__name__))
        self.__persist()

    def __persist(self):
        save_dir = os.path.join(os.path.dirname(__file__))
        weights_save_path = os.path.join(save_dir, WEIGHTS_FILE_NAME)
        model_save_path = os.path.join(save_dir, MODEL_FILE_NAME)

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        self.model.save_weights(weights_save_ath)
        with open(model_save_path, 'w') as outfile:
            json.dump(self.model.to_json(), outfile)

    def __str__(self):
        results = []
        if self.model is not None:
            results.append(self.model.summary())
        return '\n'.join(results)

In [8]:
class Basic(BaseNetwork):
    def fit(self, X_train, y_train, X_val, y_val, nb_epoch=2, batch_size=32, samples_per_epoch=None):
        output_shape = (40, 80, 3)

        print('population: ', len(X_train))

        # train model
        model = self.get_model(input_shape=output_shape, output_shape=output_shape)
        if samples_per_epoch is None:
            print('samples per epoch: {}'.format(samples_per_epoch))
            samples_per_epoch = len(X_train)

        history = model.fit_generator(
            batch_generator(X=X_train, Y=y_train, label='train set', num_epochs=nb_epoch,
                            batch_size=batch_size,
                            output_shape=output_shape,
                            classifier=self),
            nb_epoch=nb_epoch,
            samples_per_epoch=samples_per_epoch,
            validation_data=batch_generator(X_val, y_val, 'validation set', num_epochs=nb_epoch,
                                            batch_size=batch_size, output_shape=output_shape),
            verbose=2)
        
        print(history.history)
        self.save()
    
    def get_model(self, input_shape, output_shape, learning_rate=0.001, dropout_prob=0.1, activation='relu', use_weights=False):
        model = Sequential()
        model.add(Lambda(lambda x: x / 255 - 0.5,
                         input_shape=input_shape,
                         output_shape=output_shape))
        model.add(Convolution2D(24, 5, 5, border_mode='valid', activation=activation))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Convolution2D(36, 5, 5, border_mode='valid', activation=activation))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Convolution2D(48, 5, 5, border_mode='same', activation=activation))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Convolution2D(64, 3, 3, border_mode='same', activation=activation))
        model.add(Flatten())
        model.add(Dropout(dropout_prob))
        model.add(Dense(1024, activation=activation))
        model.add(Dropout(dropout_prob))
        model.add(Dense(100, activation=activation))
        model.add(Dense(50, activation=activation))
        model.add(Dense(10, activation=activation))
        model.add(Dense(1, init='normal'))

        optimizer = Adam(lr=learning_rate)
        model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        self.model = model
        model.summary()
        return model